# Issue Notebook Metadata

- Status: WIP
- Owner: hugolytics
- GitHub Issue: https://github.com/hugocool/FateForger/issues/7
- Issue Branch: issue/7-skeleton-pre-generation
- GitHub PR: https://github.com/hugocool/FateForger/pull/8
- Acceptance Criteria Reference: AC1-AC5 in tickets/skeleton_pre_generation.md and Issue #7
- Last Clean Run: 2026-02-13 16:57 UTC (markdown-only scaffold notebook)
- Environment: .venv / Python 3.11.9
- Repo Cleanliness Snapshot: dirty (`git status --porcelain`) at 2026-02-13 16:57 UTC (AGENTS/workflow updates in progress)


## Pairing Intake Record (Confirmed in Chat)

- Problem to solve: reduce ambiguity by forcing chat-first design alignment before notebook/code implementation.
- Constraints:
  - notebook-mode should be a real pairing surface, not only traceability metadata
  - human confirms direction before major coding
  - workflow remains GitHub-first for execution tracking
- Responsibility split:
  - Human: confirms problem framing, constraints, ownership boundaries, and chosen direction.
  - Agent: proposes options/tradeoffs/pseudocode, writes confirmed plan to notebook, executes modular implementation.
- Selected direction:
  - Add mandatory chat-first intake and design-options confirmation in workflow instructions.
  - Persist confirmed decisions in notebook before major coding.
- Open questions:
  - Preferred template verbosity for future `Pairing Intake Record` sections (short vs detailed).


## Design Options (Pre-Implementation Handshake)

### Option A (Recommended): Stage-local incremental extension
- Keep existing node boundaries and add focused fields/handlers.
- Tradeoff: Lowest risk and easiest review, but extends current flow complexity.
- Risks: state-flag drift unless tests are strong.
- Pseudocode outline:
  - capture stage -> if prerequisites ready: start skeleton pregen task
  - skeleton stage -> consume pregen result else fallback sync generation
  - review stage -> set pending_submit and emit confirm/cancel controls
  - confirm action -> submit, persist sync transaction, emit undo control
  - undo action -> undo last transaction, restore base snapshot, return refine

### Option B: Dedicated submission coordinator service
- Introduce service object coordinating submit/undo/pending state outside nodes.
- Tradeoff: clearer separation and future extensibility, but bigger refactor scope.
- Risks: touches more files/interfaces; higher regression risk for this ticket.
- Pseudocode outline:
  - coordinator.start_review(session) -> pending token + actions payload
  - coordinator.confirm(token) -> submit + store tx + post-submit payload
  - coordinator.undo(token) -> rollback + restore session snapshot

### Option C: Event-log-first state machine
- Model submit/undo as append-only events and derive session state from log.
- Tradeoff: strongest determinism/auditability, highest implementation cost now.
- Risks: over-engineering for current scope.
- Pseudocode outline:
  - append event(REVIEW_READY)
  - append event(SUBMIT_CONFIRMED, tx)
  - append event(UNDO_CONFIRMED, tx)
  - derive pending/committed/undone from replay

### Selected Option
- Selected: Option A
- Why: fits ticket scope AC1-AC5 with minimal churn and strong testability.


## Implementation Walkthrough / Decision Audit

### AC1: Skeleton pre-generation
- Decision: trigger background skeleton pre-generation in Stage 2 and consume in Stage 3.
- Alternatives considered:
  - Stage 3 synchronous-only draft (simpler, but slower user-perceived Stage 3)
  - separate external worker/service (more decoupled, but too much scope)
- Why chosen: minimal churn with latency reduction and deterministic fallback path.
- Inspect code:
  - `src/fateforger/agents/timeboxing/nodes/nodes.py:322`
  - `src/fateforger/agents/timeboxing/agent.py:1119`
  - `src/fateforger/agents/timeboxing/agent.py:1129`
  - `src/fateforger/agents/timeboxing/agent.py:1171`
  - `src/fateforger/agents/timeboxing/nodes/nodes.py:356`
  - `src/fateforger/agents/timeboxing/nodes/nodes.py:361`
- Inspect tests:
  - `tests/unit/test_timeboxing_skeleton_pre_generation.py:23`
  - `tests/unit/test_timeboxing_review_submit_prompt.py:72`

### AC2: Confirm before submit
- Decision: Stage 5 sets `pending_submit`; actual calendar sync only runs on explicit Slack confirm action.
- Alternatives considered:
  - auto-submit in Stage 5 (faster, but risky and less controllable)
  - modal-based confirmation (richer UI, higher complexity)
- Why chosen: explicit user control with minimal integration complexity.
- Inspect code:
  - `src/fateforger/agents/timeboxing/nodes/nodes.py:477`
  - `src/fateforger/agents/timeboxing/nodes/nodes.py:531`
  - `src/fateforger/agents/timeboxing/agent.py:1251`
  - `src/fateforger/agents/timeboxing/agent.py:2166`
  - `src/fateforger/slack_bot/timeboxing_submit.py:79`
  - `src/fateforger/slack_bot/handlers.py:1718`
- Inspect tests:
  - `tests/unit/test_timeboxing_review_submit_prompt.py:92`
  - `tests/unit/test_timeboxing_submit_flow.py:73`
  - `tests/integration/test_slack_timebox_buttons.py:149`

### AC3: Undo button flow
- Decision: expose undo only after successful submit and route via typed Slack action coordinator.
- Alternatives considered:
  - multi-level undo stack (powerful, out of scope)
  - no undo (simpler, weak safety)
- Why chosen: one-step rollback balances safety and complexity.
- Inspect code:
  - `src/fateforger/agents/timeboxing/agent.py:1259`
  - `src/fateforger/agents/timeboxing/agent.py:2255`
  - `src/fateforger/slack_bot/timeboxing_submit.py:102`
  - `src/fateforger/slack_bot/timeboxing_submit.py:179`
  - `src/fateforger/slack_bot/handlers.py:1736`
- Inspect tests:
  - `tests/unit/test_timeboxing_submit_flow.py:131`
  - `tests/integration/test_slack_timebox_buttons.py:173`

### AC4: Session memory determinism
- Decision: persist submit/undo transaction state in session (`last_sync_transaction`, `last_sync_event_id_map`) and restore `base_snapshot` on undo.
- Alternatives considered:
  - keep transaction state only inside submitter instance (not robust across turns)
  - append-only event-sourcing model (stronger, too large for this ticket)
- Why chosen: deterministic undo with local, understandable state model.
- Inspect code:
  - `src/fateforger/agents/timeboxing/agent.py:136`
  - `src/fateforger/agents/timeboxing/agent.py:2210`
  - `src/fateforger/agents/timeboxing/agent.py:2292`
  - `src/fateforger/agents/timeboxing/agent.py:2302`
- Inspect tests:
  - `tests/unit/test_timeboxing_submit_flow.py:96`
  - `tests/unit/test_timeboxing_submit_flow.py:157`
  - `tests/integration/test_slack_timebox_buttons.py:190`

### AC5: Integration coverage
- Decision: add focused integration tests around the Slack action coordinator and session transitions, with MCP mocked.
- Alternatives considered:
  - unit tests only (faster but less end-to-end confidence)
  - full live Slack E2E in CI (too brittle and environment-coupled)
- Why chosen: stable automated confidence while keeping CI deterministic.
- Inspect tests:
  - `tests/integration/test_slack_timebox_buttons.py:149`
  - `tests/integration/test_slack_timebox_buttons.py:173`
  - `tests/integration/test_slack_timebox_buttons.py:202`


## Reviewer Checklist (Decision Validation)

Use this to judge whether the chosen implementation path was correct:

1. Confirm Stage 5 no longer auto-submits.
   - Check `src/fateforger/agents/timeboxing/nodes/nodes.py:477` and test `tests/unit/test_timeboxing_review_submit_prompt.py:92`.
2. Confirm explicit confirm/cancel/undo action wiring is end-to-end.
   - Check `src/fateforger/slack_bot/timeboxing_submit.py:19`, `src/fateforger/slack_bot/handlers.py:1718`, and `tests/integration/test_slack_timebox_buttons.py:149`.
3. Confirm undo restores deterministic session state.
   - Check `src/fateforger/agents/timeboxing/agent.py:2271`, `src/fateforger/agents/timeboxing/agent.py:2302`, and `tests/unit/test_timeboxing_submit_flow.py:131`.
4. Confirm Stage 2 pre-generation and Stage 3 consume/fallback behavior.
   - Check `src/fateforger/agents/timeboxing/agent.py:1129`, `src/fateforger/agents/timeboxing/agent.py:1171`, `src/fateforger/agents/timeboxing/nodes/nodes.py:356`, and `tests/unit/test_timeboxing_skeleton_pre_generation.py:23`.
5. Re-run validation commands before merge:
   - `poetry run pytest tests/unit/test_timeboxing_skeleton_pre_generation.py tests/unit/test_timeboxing_submit_flow.py tests/unit/test_timeboxing_review_submit_prompt.py tests/integration/test_slack_timebox_buttons.py -q`
   - `poetry run pytest tests/unit/test_timeboxing_graphflow_state_machine.py tests/unit/test_phase4_rewiring.py tests/unit/test_slack_timeboxing_routing.py tests/unit/test_timeboxing_commit_skips_initial_extraction.py -q`
6. Manual Slack check remains mandatory for `User-confirmed working`.


## Open Items

- To decide:
  - whether to mark this ticket `User-confirmed working` after manual Slack verification passes.
- To do:
  - execute the `Live Slack Verification (Manual)` checklist.
  - post verification evidence/date in Issue #7 and PR #8.
  - move notebook to `notebooks/DONE/` once DoD is met.
  - switch PR #8 from draft to ready for review/merge.
- Blocked by:
  - manual Slack run in real workspace by human operator.


## Executable Walkthrough (Code Cells)

Run these cells top-to-bottom to inspect the implemented code directly from the notebook.


In [ ]:
import inspect
import subprocess
from pprint import pprint

from fateforger.agents.timeboxing.agent import Session, TimeboxingFlowAgent
from fateforger.agents.timeboxing.nodes.nodes import (
    StageCaptureInputsNode,
    StageReviewCommitNode,
    StageSkeletonNode,
)
from fateforger.slack_bot.timeboxing_submit import (
    FF_TIMEBOX_CANCEL_SUBMIT_ACTION_ID,
    FF_TIMEBOX_CONFIRM_SUBMIT_ACTION_ID,
    FF_TIMEBOX_UNDO_SUBMIT_ACTION_ID,
    build_review_submit_actions_block,
    build_undo_submit_actions_block,
)

print('imports ok')


In [ ]:
def show_source(obj, title: str, max_lines: int = 140) -> None:
    """Print source with line numbers for quick notebook review."""
    src_lines, start = inspect.getsourcelines(obj)
    end = start + len(src_lines) - 1
    print(f'\n=== {title} ({obj.__module__}.{obj.__qualname__}) lines {start}-{end} ===')
    for i, line in enumerate(src_lines[:max_lines], start=start):
        print(f'{i:4d}: {line.rstrip()}')
    if len(src_lines) > max_lines:
        print(f'... truncated to first {max_lines} lines ...')


In [ ]:
# AC1 walkthrough: Stage 2 pre-gen trigger + Stage 3 consume/fallback.
show_source(TimeboxingFlowAgent._queue_skeleton_pre_generation, 'AC1 queue pre-generation')
show_source(TimeboxingFlowAgent._consume_pre_generated_skeleton, 'AC1 consume pre-generated skeleton')
show_source(StageCaptureInputsNode.on_messages, 'AC1 Stage 2 trigger point')
show_source(StageSkeletonNode.on_messages, 'AC1 Stage 3 consume point')


In [ ]:
# AC2/AC3/AC4 walkthrough: submit gate + confirm/cancel/undo handlers.
show_source(StageReviewCommitNode.on_messages, 'AC2 Stage 5 sets pending_submit')
show_source(TimeboxingFlowAgent._render_submit_prompt_blocks, 'AC2 prompt block rendering')
show_source(TimeboxingFlowAgent.on_confirm_submit, 'AC2 confirm submit handler')
show_source(TimeboxingFlowAgent.on_cancel_submit, 'AC2 cancel submit handler')
show_source(TimeboxingFlowAgent.on_undo_submit, 'AC3/AC4 undo submit handler')
show_source(build_review_submit_actions_block, 'AC2 Slack confirm/cancel block')
show_source(build_undo_submit_actions_block, 'AC3 Slack undo block')


In [ ]:
# Lightweight behavior check: generated block payloads and action IDs.
session = Session(thread_ts='t1', channel_id='c1', user_id='u1')
agent = TimeboxingFlowAgent.__new__(TimeboxingFlowAgent)
meta_value = agent._build_timeboxing_action_value(session)

review_block = build_review_submit_actions_block(meta_value=meta_value)
undo_block = build_undo_submit_actions_block(meta_value=meta_value)

pprint(review_block)
pprint(undo_block)

review_ids = [el['action_id'] for el in review_block['elements']]
undo_ids = [el['action_id'] for el in undo_block['elements']]

assert FF_TIMEBOX_CONFIRM_SUBMIT_ACTION_ID in review_ids
assert FF_TIMEBOX_CANCEL_SUBMIT_ACTION_ID in review_ids
assert FF_TIMEBOX_UNDO_SUBMIT_ACTION_ID in undo_ids
print('action ID smoke checks passed')


In [ ]:
# Optional: run the ticket-focused tests from the notebook.
cmd = (
    '~/.local/pipx/venvs/poetry/bin/poetry run pytest '
    'tests/unit/test_timeboxing_skeleton_pre_generation.py '
    'tests/unit/test_timeboxing_submit_flow.py '
    'tests/unit/test_timeboxing_review_submit_prompt.py '
    'tests/integration/test_slack_timebox_buttons.py -q'
)
result = subprocess.run(cmd, shell=True, check=False, capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print(result.stderr)
print(f'pytest exit code: {result.returncode}')


## Acceptance Criteria Checklist

- [x] AC1 Skeleton pre-generation in Stage 2 + Stage 3 consume/fallback path.
- [x] AC2 Confirm-before-submit review flow with Slack buttons.
- [x] AC3 Undo flow with post-submit undo button and handler.
- [x] AC4 Deterministic session memory transitions around submit/undo.
- [x] AC5 Integration tests for Slack button flows.
- [ ] Human verification in live Slack (required for `User-confirmed working`).


## Live Slack Verification (Manual)

Run this in the real Slack workspace and fill results:

1. Start a timeboxing session and reach Stage 5 review.
2. Confirm UI shows both buttons:
   - `✅ Submit to Calendar`
   - `↩️ Keep Editing`
3. Click `↩️ Keep Editing`.
   - Expected: pending submit clears and flow returns to Refine stage.
4. Click `✅ Submit to Calendar`.
   - Expected: calendar submit succeeds and message shows Undo button.
5. Click `↩️ Undo`.
   - Expected: rollback succeeds, base snapshot restored, flow returns to Refine stage.
6. End session and attempt Undo again.
   - Expected: rejected with ended-session guard.

Record evidence:
- Date/time:
- Workspace/channel/thread:
- Result: pass/fail
- Notes/screenshots links:


## Implementation Evidence

- Main implementation commit: `36182ba`.
- Ticket source: `tickets/skeleton_pre_generation.md`.
- Key module touchpoints:
  - `src/fateforger/agents/timeboxing/agent.py`
  - `src/fateforger/agents/timeboxing/nodes/nodes.py`
  - `src/fateforger/slack_bot/handlers.py`
  - `src/fateforger/slack_bot/timeboxing_submit.py`
- Validation command sets executed:
  - `poetry run pytest tests/unit/test_timeboxing_skeleton_pre_generation.py tests/unit/test_timeboxing_submit_flow.py tests/unit/test_timeboxing_review_submit_prompt.py tests/integration/test_slack_timebox_buttons.py -q`
  - `poetry run pytest tests/unit/test_timeboxing_graphflow_state_machine.py tests/unit/test_phase4_rewiring.py tests/unit/test_slack_timeboxing_routing.py tests/unit/test_timeboxing_commit_skips_initial_extraction.py -q`


## Extraction Map (Notebook -> Artifacts)

| Notebook concern | Durable artifact destination |
| --- | --- |
| Skeleton pre-generation behavior | `src/fateforger/agents/timeboxing/agent.py`, `src/fateforger/agents/timeboxing/nodes/nodes.py` |
| Confirm/undo submit workflow | `src/fateforger/agents/timeboxing/agent.py`, `src/fateforger/agents/timeboxing/nodes/nodes.py`, `src/fateforger/slack_bot/handlers.py`, `src/fateforger/slack_bot/timeboxing_submit.py` |
| Session state persistence for submit/undo | `src/fateforger/agents/timeboxing/agent.py` |
| Deterministic behavior checks | `tests/unit/test_timeboxing_skeleton_pre_generation.py`, `tests/unit/test_timeboxing_submit_flow.py`, `tests/unit/test_timeboxing_review_submit_prompt.py` |
| End-to-end Slack flow checks | `tests/integration/test_slack_timebox_buttons.py` |
| Human-readable execution trace | GitHub Issue #7 + PR #8 checkpoints |
| Product/knowledge context | Notion link from Issue/PR (if maintained) |


## Closeout / Remaining Notebook-Only Content

- This notebook is now a lightweight traceability scaffold (no duplicated production code cells).
- Remaining step before moving to `notebooks/DONE/`: perform live Slack verification and record confirmation date.
- After PR closure, keep this notebook only as a concise execution record or move to `notebooks/DONE/` per workflow decision.


## Stage 3 Agent Inspection Entry Points
Use these cells to inspect the real implementation paths (no duplicated notebook logic) and run the Stage 3 flow through the actual agent.


In [ ]:
from fateforger.agents.timeboxing.notebook_entrypoints import (
    create_agent,
    create_session,
    stage3_method_locations,
    stage3_framework_report,
    stage3_source_snippets,
    run_stage3_draft,
    run_graph_turn,
)


In [ ]:
# Exact file/line entrypoints for Stage 3 path
stage3_method_locations()


In [ ]:
# Quick check: are we using framework-native path vs post-hoc hacks?
stage3_framework_report()


In [ ]:
# Inspect source snippets of the real methods
snips = stage3_source_snippets()
print(snips['agent._run_skeleton_draft'][:2500])
print('\n' + '=' * 80 + '\n')
print(snips['node.StageSkeletonNode.on_messages'][:2500])


In [ ]:
# Create real agent + real session object
agent = create_agent()
session = create_session()
session


In [ ]:
# Run Stage 3 draft path directly
import asyncio
draft_trace = asyncio.run(run_stage3_draft(agent=agent, session=session))
draft_trace


In [ ]:
# Talk to it through one real GraphFlow turn
session.stage = session.stage.SKELETON
turn_trace = asyncio.run(run_graph_turn(agent=agent, session=session, user_text='proceed'))
turn_trace.response_text


## 2026-02-14 Stage 3/4 Shared Policy Decision

Confirmed direction for Issue #7 extension:
- Shared policy core is reused by both Stage 3 and Stage 4 prompts.
- Stage 3 remains markdown-first and concise (coarse durations + fixed anchors).
- Stage 4 keeps full refinement behavior with macro/micro guidance and quality scoring facts (0-4, advisory only).
- Quality evaluation is LLM-driven via AutoGen typed output; no deterministic text/keyword heuristics.
- Stage 4 continues sync-on-success per refine turn.


## AC Mapping Update (Issue #7 Extension)

- Shared policy reuse:
  - `src/fateforger/agents/timeboxing/planning_policy.py`
  - `src/fateforger/agents/timeboxing/prompt_rendering.py`
  - `src/fateforger/agents/timeboxing/skeleton_draft_system_prompt.j2`
  - `src/fateforger/agents/timeboxing/patching.py`
- Stage 3 output contract:
  - `src/fateforger/agents/timeboxing/skeleton_draft_system_prompt.j2`
  - `src/fateforger/agents/timeboxing/agent.py` (`_tb_plan_overview_markdown`)
- Stage 4 quality facts (advisory):
  - `src/fateforger/agents/timeboxing/stage_gating.py`
  - `src/fateforger/agents/timeboxing/agent.py` (`RefineQualityFacts`, `_run_refine_quality_assessment`)
  - `src/fateforger/agents/timeboxing/nodes/nodes.py`
- Targeted tests:
  - `tests/unit/test_timeboxing_prompt_rendering.py`
  - `tests/unit/test_patching.py`
  - `tests/unit/test_timeboxing_skeleton_draft_contract.py`
  - `tests/unit/test_phase4_rewiring.py`
  - `tests/unit/test_timeboxing_stage3_markdown_block.py`

Open Items
- To decide: none
- To do: run targeted pytest set and post progress checkpoint to Issue/PR
- Blocked by: none
